##### <p> Samuel Wolfe <br> August 1, 2023 <br> MSBA 206 <br> DMBA SBA Case 1</p>

In [ ]:
%matplotlib inline
from pathlib import Path
import pandas as pd
import requests
import io
import matplotlib.pylab as plt
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.naive_bayes import MultinomialNB, CategoricalNB
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn.linear_model import LassoCV, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from dmba import regressionSummary, classificationSummary, liftChart, gainsChart
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [ ]:
def readFile(url):
    download = requests.get(url).content
    # Reading the downloaded content and turning it into a pandas dataframe
    df = pd.read_csv(io.StringIO(download.decode('utf-8')))
    return df
def statslist(df):
    dfStats = pd.DataFrame({'Mean' : df.mean(numeric_only=True),
            'SD' : df.std(numeric_only=True),
            'Min' : df.min(),
            'Max' : df.max(),
            'Median' : df.median(numeric_only=True),
            })
    return dfStats
def categorize(df):
    for x in df:
        df[x] = df[x].astype('category')
    return df

In [ ]:
dfVote = readFile("https://raw.githubusercontent.com/wolfesamk/MSBA-206/main/dmba/Voter-Persuasion.csv")
 #do not need these as they are duplicates of values or states as not useable
dropCol = ['VOTER_ID','SET_NO','Partition','opposite','MOVED_AD']
dfVote = dfVote.drop(columns=dropCol)
outcome = 'MOVED_A'

In [ ]:
X = pd.get_dummies(dfVote.drop(columns=['MOVED_A','COMM_PT']), drop_first=True)
predictors = X.columns.to_list()
y = dfVote[outcome]
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.4, random_state=0)

In [ ]:
clf = LassoCV().fit(X, y)
importance = np.abs(clf.coef_)

In [ ]:
idx_third = importance.argsort()[-3]
threshold = importance[idx_third] + 0.01

idx_features = (-importance).argsort()[:10]
name_features = np.array(predictors)[idx_features]
print('Selected features: {}'.format(name_features))

sfm = SelectFromModel(clf, threshold=threshold)
sfm.fit(X, y)
X_transform = sfm.transform(X)

n_features = sfm.transform(X).shape[1]